##🗂️ 1. Kết nối Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qqq langchain==0.3.4 qdrant-client==1.12.0 langchain-community==0.3.3 langchain-huggingface==0.1.0 gradio pyngrok fastapi uvicorn -U langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.4/266.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 k

In [ ]:
import uvicorn
import threading
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Qdrant
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time
from fastapi.middleware.cors import CORSMiddleware

# Thiết lập các thông số cấu hình
EMBEDDINGS_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
QDRANT_URL = "https://5b4dc1d1-411b-4080-b7ee-af63f425703a.us-east4-0.gcp.cloud.qdrant.io"
QDRANT_COLLECTION_NAME = "humg_data_vector_db_v1"
HUGGINGFACE_API_KEY = "hf_NwKvEZkxWnXneGQFZDSHOzkEafDLce"
QDRANT_API_KEY = "e69LIRXuJksyI3QJfA6jy6RabMGLYf9EJ6zW8RqdEgn5TNmGTJkKFQ"
ngrok.set_auth_token("2Zj2TcUYcuHaBGfmjdEpJ2wNx9w_2XxAgmh3cgKpx3d7uiwX2")  # Đặt mã ngrok của bạn ở đây

# Khởi tạo kết nối Qdrant và Embeddings
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, prefer_grpc=False)
db = Qdrant(client=client, embeddings=embeddings, collection_name=QDRANT_COLLECTION_NAME)
retriever = db.as_retriever(search_kwargs={"k": 10})

# Tải mô hình từ Google Drive
MODEL_PATH = '/content/drive/MyDrive/HUMG Chatbot/Vi-Qwen2-7B'
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Tạo pipeline sinh văn bản
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024)
hf_pipeline = HuggingFacePipeline(pipeline=pipe)

# Thiết lập RAG (Retrieval-Augmented Generation)
query_template = """Bạn là một chatbot thông minh trả lời câu hỏi dựa trên ngữ cảnh (context).\n\n### Context:{context} \n\n### Human: {question}\n\n### Assistant:"""
prompt = PromptTemplate(template=query_template, input_variables=["context", "question"])
rag_pipeline = RetrievalQA.from_chain_type(
    llm=hf_pipeline,
    chain_type='stuff',
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# Hàm trả lời câu hỏi
def get_response(question: str) -> str:
    retrieved_docs = retriever.get_relevant_documents(question)
    context = " ".join([doc.page_content for doc in retrieved_docs])

    conversation = [{"role": "system", "content": "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."}]
    conversation.append({"role": "user", "content": f"### Context:\n{context}\n\n### Question:\n{question}\n\n### Answer:"})

    text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer(text, return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=1024, temperature=0.3)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# Định nghĩa mô hình dữ liệu đầu vào
class QuestionRequest(BaseModel):
    question: str


<ipython-input-3-b1bafb5e4f6b>:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-3-b1bafb5e4f6b>:29: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  db = Qdrant(client=client, embeddings=embeddings, collection_name=QDRANT_COLLECTION_NAME)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-3-b1bafb5e4f6b>:44: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_pipeline = HuggingFacePipeline(pipeline=pipe)


In [ ]:
origins = ["*"]
# Khởi tạo FastAPI
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Endpoint API để nhận câu hỏi và trả về câu trả lời

@app.get("/")
def read_root():
    return "API RAG by EpChannel"

@app.post("/ask/")  # hoặc sử dụng @app.api_route("/ask/", methods=["GET", "POST"]) nếu muốn hỗ trợ cả hai
async def ask(request: QuestionRequest):
    answer = get_response(request.question)
    return {"question": request.question, "answer": answer}

# Kết nối trực tiếp tới domain tĩnh
public_url = ngrok.connect(addr=8000, hostname="lark-discrete-slug.ngrok-free.app")  # Thay bằng domain tĩnh của bạn
print(f"Public URL: {public_url.public_url}")

# Khởi động server FastAPI trên một luồng riêng
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Khởi chạy server trong một luồng riêng
server_thread = threading.Thread(target=start_server)
server_thread.start()

# Vòng lặp vô hạn để giữ cho cell chạy liên tục
while True:
    time.sleep(1)


INFO:     Started server process [1173]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://lark-discrete-slug.ngrok-free.app
INFO:     113.185.48.81:0 - "OPTIONS /ask/ HTTP/1.1" 200 OK


<ipython-input-3-b1bafb5e4f6b>:59: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


INFO:     113.185.48.81:0 - "POST /ask/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 165, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-pack

INFO:     113.185.48.81:0 - "POST /ask/ HTTP/1.1" 200 OK


KeyboardInterrupt: 

In [ ]:
!pip freeze > requirements.txt


In [ ]:
!pkill ngrok
!fuser -k 8000/tcp


In [ ]:
import pandas as pd
import requests
import time

# Tải dataset từ file cục bộ
dataset = pd.read_csv('datasethumg.csv')

# API Endpoint
API_URL = "http://localhost:8000/ask/"
K = 10  # Top k kết quả
MAX_QUESTIONS = 50  # Giới hạn số câu hỏi để kiểm thử nhanh

# Hàm gửi câu hỏi và nhận kết quả
def get_context_from_api(question):
    try:
        start_time = time.time()
        response = requests.post(API_URL, json={"question": question})  # Timeout đã bị loại bỏ
        end_time = time.time()

        if response.status_code == 200:
            data = response.json()
            context = data.get("answer", "")  # Giả định context nằm trong 'answer'
            response_time = end_time - start_time
            return context.split("\n")[:K], response_time  # Lấy top K dòng đầu tiên
        else:
            print(f"API Error: {response.status_code}")
            return [], None
    except Exception as e:
        print(f"Request Error: {e}")
        return [], None

# Tính Precision@k và Recall@k
def evaluate_retrieval_accuracy(dataset, k=10, max_questions=50):
    precision_scores = []
    recall_scores = []
    response_times = []

    for index, row in dataset.head(max_questions).iterrows():
        question = row['anchor']
        positive = row['positive']

        print(f"Evaluating question {index+1}/{max_questions}: {question}")
        retrieved_contexts, response_time = get_context_from_api(question)

        if response_time is not None:
            response_times.append(response_time)

        if not retrieved_contexts:
            print(f"No context retrieved for question {index+1}")
            continue

        # Đếm số context phù hợp
        relevant_count = sum(1 for ctx in retrieved_contexts if positive in ctx)

        # Precision@k
        precision = relevant_count / min(k, len(retrieved_contexts))
        precision_scores.append(precision)

        # Recall@k (Giả định chỉ có 1 ngữ cảnh phù hợp)
        recall = relevant_count / 1
        recall_scores.append(recall)

    avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    avg_response_time = sum(response_times) / len(response_times) if response_times else 0

    return avg_precision, avg_recall, avg_response_time

# Đo lường độ chính xác truy xuất
start_time = time.time()
precision, recall, avg_response_time = evaluate_retrieval_accuracy(dataset, k=K, max_questions=MAX_QUESTIONS)
end_time = time.time()

# Kết quả
results = {
    "Precision@k": precision,
    "Recall@k": recall,
    "Average Response Time (seconds)": avg_response_time,
    "Total Questions Evaluated": MAX_QUESTIONS,
    "Time Taken (seconds)": end_time - start_time
}

print("\n=== Evaluation Results ===")
for key, value in results.items():
    print(f"{key}: {value}")
